# Deutch Algorithm

Given a function $f(x)$ where $x = 0$ or $1$ and $f(x) = 0$ or $1$, we want to decide whether $f(x)$ is **constant** or **balanced**.

* **constant** means that $f(0) = f(1)$
* **balanced** means that $f(0) \neq f(1)$

In [1]:
from quantum import *

def constant(x): return 0
def balanced(x): return x

TARGET_FUNC = constant

Start with the $\left|00\right\rangle$ state.

In [2]:
state = Q(kron(zero, zero))
print(state)

[[1]
 [0]
 [0]
 [0]]


Apply `X` to all qubits.

In [3]:
for i in range(state.num_qubits):
    state = state.apply_gate(X, i)
print(state)

[[ 0.]
 [ 0.]
 [ 0.]
 [ 1.]]


Apply `H` to all qubits.

In [4]:
for i in range(state.num_qubits):
    state = state.apply_gate(H, i)
print(state)

[[ 0.5+0.j]
 [-0.5+0.j]
 [-0.5+0.j]
 [ 0.5+0.j]]


Apply the function $f(x)$ with the unitary operator $U\left|x\right\rangle\left|y\right\rangle = \left|x\right\rangle\left|y \oplus f(x)\right\rangle$

In [5]:
def apply_func(Qstate, func):
    qubits = range(2 ** Qstate.num_qubits)
    #print("state  (bin)   x    y    y^f(x)")
    state_matrix = Qstate.state
    swapped = set([])
    for current_state in qubits:
        x = current_state >> 1  # all bits n-1
        y = current_state & 1   # only bit 0
        yfunc = y ^ func(x)
        #print("{:5}   {:>4} {:>4} {:>4}   {:>4}".format(current_state, bin(current_state), bin(x), bin(y), bin(yfunc)))
        # the new state is the original `x` bits with the `y ^ f(x)` bit at the end
        new_state = (x << 1) + yfunc
        if (current_state, new_state) not in swapped:
            state_matrix = row_swap(state_matrix, current_state, new_state)
            swapped.add((current_state, new_state))
            swapped.add((new_state, current_state))
    return Q(state_matrix)

state = apply_func(state, TARGET_FUNC)
print(state)

[[ 0.5+0.j]
 [-0.5+0.j]
 [-0.5+0.j]
 [ 0.5+0.j]]


Perform a `H` on the last (1st) qubit.

In [6]:
state = state.apply_gate(H, 1)
print(state)

[[ 0.00000000+0.j]
 [ 0.00000000+0.j]
 [ 0.70710678+0.j]
 [-0.70710678+0.j]]


We measure the state and look at the value of the last qubit.

* If $f(x)$ is constant, then the last qubit will always be 1.
* If $f(x)$ is balanced, then the last qubit will always be 0.

In [7]:
TRIALS = 1000
outcomes = {}
for i in range(TRIALS):
    measurement = state.measure()
    outcomes[measurement] = outcomes.get(measurement, 0) + 1

qubit1_outcomes = {}
for (outcome, occurrences) in outcomes.items():
    qubit1 = outcome >> 1
    qubit1_outcomes[qubit1] = qubit1_outcomes.get(qubit1, 0) + occurrences

print("The target function is {}.".format(TARGET_FUNC.__name__))
for (outcome, occurrences) in qubit1_outcomes.items():
    if outcome == 1: result = "constant"
    elif outcome == 0: result = "balanced"
    print("Last qubit measured as {outcome}, therefore {result}. ({occurrences} occurrences = {success:.2f}%)".format(
        outcome=outcome, result=result, occurrences=occurrences, success=(occurrences*100/TRIALS))
    )

The target function is constant.
Last qubit measured as 1, therefore constant. (1000 occurrences = 100.00%)
